In [2]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
#read initial csv into a dataframe
csv_file = 'Main Avocado Tree (Resources)/avocado-updated-2020.csv'
avocado_df = pd.read_csv(csv_file)
avocado_df.head()

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.0,organic,2015,Albany
2,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.0,conventional,2015,Atlanta
3,2015-01-04,1.76,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.0,organic,2015,Atlanta
4,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.0,conventional,2015,Baltimore/Washington


In [ ]:
# Question 1: Which cities love avocados most?
#Task 1: popularity of avocado, visual heatmap, 2015 to 2020, bar graph of volume per area 

In [ ]:
#Question 2: Do states with higher income per capita buy more avocados? (income per capita from 2017-2019)
#Task 2: line graph (x-values states, y-values income per capita and avocado revenue) 

In [ ]:
#Question 3: Which type of avocado is the biggest source of revenue for suppliers? 
#Task 3: Five pie charts by type (value = revenue) for each year on one png 

In [ ]:
#Question 4: Is there a time of year that sees a spike in revenue?
#Task 4: bar chart (x-values months, y-values avocado revenue)

In [ ]:
#Question 5: How has the average avocado price changed over time?  What is the predicted avocado price for 2021 based on our data?  
#Task 5: scatter plot with regression to predict 2021 average price 

In [ ]:
#Question 6: Do different types of avocados have different price trends?
#Task 6: facet (multiple lines on one) line graph (x-values = year, y-values = average price) for each type